# Example 1: Sinatra

In [1]:
from youtube_utils import make_task, transcribe_video

In [2]:
url = 'https://www.youtube.com/watch?v=ZEcqHA7dbwM'
model_type = 'tiny'
transcription_task = make_task(url, model_type)

In [3]:
transcription = transcribe_video(transcription_task)
transcription

Loading tiny model...
Model loaded succssfully...
Transcribing ./youtube-flow-audio-files/fly_me_to_the_moon_2008_remastered...


/Users/eddie/miniconda3/envs/youtube-transcription/lib/python3.10/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


' Let me play among the stars. Let me see what spring is like on. As you put a mask, in other words, hold my hand. In other words, baby kiss me. Fill my heart with song and let me sing forevermore. You are all I long for, all I worship and the dawn. In other words, please be true. In other words, I love you. Fill my heart with song and let me sing forevermore. You are all I long for, all I worship and the dawn. In other words, please be true. In other words, in other words, I love you. You.'

# Example 2: Fireside Chat #1
* Video Time: 02:21:10

In [5]:
%%time
url = 'https://www.youtube.com/watch?v=Dr6DsWa6Dhg'
model_type = 'tiny'
transcription_task = make_task(url, model_type)
fs_chat_transcription = transcribe_video(transcription_task)
fs_chat_transcription

Loading tiny model...
Model loaded succssfully...
Transcribing ./youtube-flow-audio-files/fireside_chat_1_how_to_produce_sustainable_business_value_with_machine_learning...


/Users/eddie/miniconda3/envs/youtube-transcription/lib/python3.10/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

# Running Flows

## Transcribe one Video

In [20]:
! python youtube_video_transcriber.py run --url 'https://www.youtube.com/watch?v=ZEcqHA7dbwM'

Metaflow 2.7.11 executing YouTubeVideoTranscription for user:eddie
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-10-08 18:56:08.964 Workflow starting (run-id 184193):
2022-10-08 18:56:10.591 [184193/start/1001752 (pid 64624)] Task is starting.
2022-10-08 18:56:18.503 [184193/start/1001752 (pid 64624)] Foreach yields 1 child steps.
2022-10-08 18:56:18.503 [184193/start/1001752 (pid 64624)] Task finished successfully.
2022-10-08 18:56:21.215 [184193/transcribe/1001753 (pid 64634)] Task is starting.
2022-10-08 18:56:23.142 [184193/transcribe/1001753 (pid 64634)] [pod t-vll8s-k8n7b] Task is starting (Pod is pending, Container is waiting - ContainerCreating)...
2022-10-08 18:56:53.238 [184193/transcribe/1001753 (pid 64634)] [pod t-vll8s-k8n7b] Task is starting (Pod is pending, Container is waiting - ContainerCreating)...
2022-10-08 18:57:23.913 [184193/transcribe/1001753 (pid 64634)] [pod t-vll8s-k8n7b] Task is starting (Pod is pending, Contai

## Transcribe each Video in a Playlist

[This url](https://www.youtube.com/playlist?list=PLUsOvkBBnJBc1fcDQEOPJ77pMcE4CnNxc) goes to the playlist for Ville's [tagging blog](/blog/five-ways-to-use-the-new-metaflow-tags/). The playlist consists of 5 videos:
* [Basic Tagging](https://www.youtube.com/watch?v=DEmKaTI3MG4&list=PLUsOvkBBnJBc1fcDQEOPJ77pMcE4CnNxc&index=1): 05:41
* [Programmatic Tagging](https://www.youtube.com/watch?v=25Hqp43J37I&list=PLUsOvkBBnJBc1fcDQEOPJ77pMcE4CnNxc&index=2): 04:52
* [Tags and Namespaces](https://www.youtube.com/watch?v=ifARsmiSNhE&list=PLUsOvkBBnJBc1fcDQEOPJ77pMcE4CnNxc&index=3): 10:34
* [Tags in CI/CD](https://www.youtube.com/watch?v=hIiDXPHqEFM&list=PLUsOvkBBnJBc1fcDQEOPJ77pMcE4CnNxc&index=4): 03:28
* [Tags and Continuous Training](https://www.youtube.com/watch?v=lZhwhuG0AN8&list=PLUsOvkBBnJBc1fcDQEOPJ77pMcE4CnNxc&index=5): 04:33

In [10]:
! python youtube_video_transcriber.py run \
    --url 'https://www.youtube.com/playlist?list=PLUsOvkBBnJBc1fcDQEOPJ77pMcE4CnNxc' \
    --m 'large'

Traceback (most recent call last):
  File "/Users/eddie/Dev/operationalizing-whisper/youtube_video_transcriber.py", line 11, in <module>
    class YouTubeVideoTranscription(FlowSpec):
  File "/Users/eddie/Dev/operationalizing-whisper/youtube_video_transcriber.py", line 51, in YouTubeVideoTranscription
    flag = int(os.getenv('IS_REMOTE'))
TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


In [107]:
run = Flow('VideoTranscription').latest_successful_run
run.data.results

,url,filename,model_type,title,author,views,length (s),description,id,publish date,transcribed_text
0,https://youtube.com/watch?v=hIiDXPHqEFM,metaflow_tags_tags_in_cicd,small,Metaflow Tags: Tags in CI/CD,Outerbounds,45,208,Learn how to use Metaflow tags in conjunction ...,hIiDXPHqEFM,2022-06-21,Let's turn into more production-oriented use ...
1,https://youtube.com/watch?v=lZhwhuG0AN8,metaflow_tags_tags_and_continuous_training,small,Metaflow Tags: Tags and Continuous Training,Outerbounds,27,272,Learn how to use tags to facilitate continuous...,lZhwhuG0AN8,2022-06-21,This last example might be one of the most si...
2,https://youtube.com/watch?v=25Hqp43J37I,metaflow_tags_programmatic_tagging,small,Metaflow Tags: Programmatic Tagging,Outerbounds,31,291,Learn how to use Metaflow tags programmaticall...,25Hqp43J37I,2022-06-21,In some sense this example is an inverse of t...
3,https://youtube.com/watch?v=ifARsmiSNhE,metaflow_tags_tags_and_namespaces,small,Metaflow Tags: Tags and Namespaces,Outerbounds,17,633,Learn how to use Metaflow tags to create isola...,ifARsmiSNhE,2022-06-21,"From the last two examples, you may have got ..."
4,https://youtube.com/watch?v=DEmKaTI3MG4,metaflow_tags_basic_tagging,small,Metaflow Tags: Basic Tagging,Outerbounds,69,341,Learn how to use Metaflow tags for experiment ...,DEmKaTI3MG4,2022-06-21,Let's start with the very simple use case for...


## Transcribe a List of Videos

In [ ]:
! python youtube_video_transcriber.py run --urls 'science_video_urls.txt'

# Analysis

In [6]:
import humanize
import datetime as dt
from metaflow import Flow
run = Flow('YouTubeVideoTranscription').latest_successful_run
humanize.naturaltime(dt.datetime.now() - run.created_at)

'17 minutes ago'

In [7]:
run.data.results

,url,filename,model_type,title,author,views,length,description,id,publish_date,transcription_text
0,https://youtube.com/watch?v=GVsUOuSjvcg,future_computers_will_be_radically_different_a...,tiny,Future Computers Will Be Radically Different (...,Veritasium,8672686,1302,Visit https://brilliant.org/Veritasium/ to get...,GVsUOuSjvcg,2022-03-01,"For hundreds of years, analog computers were ..."
1,https://youtube.com/watch?v=ieDIpgso4no,breaking_new_phase_of_matter,tiny,BREAKING: New Phase of Matter,Physics Girl,1445568,934,What are time crystals? How do scientists make...,ieDIpgso4no,2022-06-15,We're filming a video about time crystals. Ok...
2,https://youtube.com/watch?v=Ek1buV2HA68,worlds_only_moving_mud_puddle,tiny,World's Only Moving Mud Puddle,Physics Girl,4100457,699,Want to support more videos like this? Patreon...,Ek1buV2HA68,2021-05-10,"Hey, let's pull them straight in line in here..."
3,https://youtube.com/watch?v=ccO0_hSXMyM,how_mirrors_could_solve_our_energy_problem,tiny,How Mirrors Could Solve our Energy Problem,Physics Girl,1072714,690,We visited a giant field of solar mirrors to l...,ccO0_hSXMyM,2021-08-31,"This is 4,000 acres of mirrors. As you drive ..."
4,https://youtube.com/watch?v=S2xHZPH5Sng,clickbait_is_unreasonably_effective,tiny,Clickbait is Unreasonably Effective,Veritasium,6098423,1165,The title and thumbnail play a huge role in a ...,S2xHZPH5Sng,2021-08-17,Can I tell you something I'm bad at? I am ter...
5,https://youtube.com/watch?v=HeQX2HjkcNo,maths_fundamental_flaw,tiny,Math's Fundamental Flaw,Veritasium,21846330,2039,Not everything that is true can be proven. Thi...,HeQX2HjkcNo,2021-05-22,There is a hole at the bottom of math. A hole...


In [8]:
run.data.results.transcribed_text.values[0].strip()

AttributeError: 'DataFrame' object has no attribute 'transcribed_text'

# Doing Random Stuff

## Grab a YouTube Video

In [18]:
from pytube import YouTube

In [19]:
video = YouTube('https://youtube.com/watch?v=hIiDXPHqEFM')

# Notes
* `base` model on remote instances produced `RuntimeError: Model has been downloaded but the SHA256 checksum does not not match. Please retry loading the model.` [link](https://github.com/openai/whisper/blob/82725cea9c339cdc3a2004a622bba766b1871945/whisper/__init__.py#L58). Error was on batch cpu compute environment. 